In [1]:
# torch-related imports
import torch
from torch import nn, Tensor
from torch.autograd import Variable
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import DataLoaderIter
from torch.utils.data.dataset import Dataset
from torchvision.transforms import Compose, RandomRotation, RandomResizedCrop, ToTensor

# helper functions
from os.path import join

# full libraries
import pandas as pd
import numpy as np
import PIL
import matplotlib.pyplot as plt

# set random seeds
seed = 22
np.random.seed(seed)
_ = torch.random.manual_seed(seed)

In [2]:
class FlickrDataset(Dataset):
    def __init__(self, annotations_path, images_path, transform):
        self.images_path = images_path
        self.df = pd.read_csv(annotations_path,
                              sep=" ",
                              header=None,
                              names=["file_name", "logo", 1, 2, 3, 4, 5, 6])
        self.labels_dict = {label: idx for idx, label in enumerate(self.df["logo"].unique())}
        self.transform = transform

    def __getitem__(self, index):
        row = self.df.loc[index, :]
        fpath = join(self.images_path, row["file_name"])
        im = PIL.Image.open(fpath)
        return self.transform(im), self.labels_dict[row["logo"]]

    def __len__(self):
        return self.df.shape[0]

In [3]:
flickrData = FlickrDataset(join("data", "flickr_logos_27_dataset", "flickr_logos_27_dataset_training_set_annotation.txt"),
                          join("data", "flickr_logos_27_dataset", "flickr_logos_27_dataset_images"),
                          transform=Compose([
                              RandomRotation(10),
                              RandomResizedCrop(256),
                              ToTensor()
                          ]))

In [4]:
batch_size = 16
flickrLoader = DataLoader(flickrData,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=2,
                          drop_last=True)

In [5]:
class MyClassifier(nn.Module):
    def __init__(self):
        super(MyClassifier, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 4, kernel_size=3, stride=2),
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        print(self.layer1)
        self.layer2 = nn.Sequential(
            nn.Conv2d(4, 4, kernel_size=3, stride=2),
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        print(self.layer2)
        self.linear1 = nn.Sequential(
            nn.Linear(900, 27, bias=True),
            nn.ReLU()
        )
        print(self.linear1)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.linear1(out)
        return out

In [ ]:
classifier = MyClassifier()
learning_rate = 0.0001
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

Sequential(
  (0): Conv2d (3, 4, kernel_size=(3, 3), stride=(2, 2))
  (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1))
)
Sequential(
  (0): Conv2d (4, 4, kernel_size=(3, 3), stride=(2, 2))
  (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1))
)
Sequential(
  (0): Linear(in_features=900, out_features=27)
  (1): ReLU()
)


In [ ]:
num_epochs = 100
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(DataLoaderIter(flickrLoader)):
        images = Variable(images)
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = classifier(images)
        l = loss(outputs, labels)
        l.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(flickrData)//batch_size, l.data[0]))


Epoch [1/100], Iter [100/283] Loss: 3.3995
Epoch [1/100], Iter [200/283] Loss: 3.1441
Epoch [2/100], Iter [100/283] Loss: 3.2050
Epoch [2/100], Iter [200/283] Loss: 3.2897
Epoch [3/100], Iter [100/283] Loss: 2.9070
Epoch [3/100], Iter [200/283] Loss: 3.0633
Epoch [4/100], Iter [100/283] Loss: 2.7599
Epoch [4/100], Iter [200/283] Loss: 2.8744
Epoch [5/100], Iter [100/283] Loss: 3.0504
Epoch [5/100], Iter [200/283] Loss: 2.9060
Epoch [6/100], Iter [100/283] Loss: 2.5182
Epoch [6/100], Iter [200/283] Loss: 2.9077
Epoch [7/100], Iter [100/283] Loss: 2.9936
Epoch [7/100], Iter [200/283] Loss: 3.0463
Epoch [8/100], Iter [100/283] Loss: 2.9636
Epoch [8/100], Iter [200/283] Loss: 2.7774
Epoch [9/100], Iter [100/283] Loss: 3.0437
Epoch [9/100], Iter [200/283] Loss: 2.8595
Epoch [10/100], Iter [100/283] Loss: 2.6458
Epoch [10/100], Iter [200/283] Loss: 2.7322
Epoch [11/100], Iter [100/283] Loss: 2.8958
Epoch [11/100], Iter [200/283] Loss: 3.3333
Epoch [12/100], Iter [100/283] Loss: 3.2802
Epoch 